In [1]:
"""
This ipython notebook contains the code to create and train the proposed model using the theano. 
"""
import gensim
import pickle
import random
import sys
import time

import numpy as np
from nltk.corpus import stopwords
import scipy.stats as meas
from collections import OrderedDict
from scipy.optimize import linprog

import theano.tensor as T
import theano.tensor as tensor
from theano import config
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams

def _p(pp, name):
    return '%s_%s' % (pp, name)
    import re

Using Theano backend.


In [1]:
"""
Loading pretrained 300d word2vec (On google news corpus) using gensim
"""
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin",binary=True)

Using Theano backend.


MemoryError: 

In [ ]:
#Defining functions for preprocessing raw text

def prepare_data(data, add=0.0, mul=2.0):
    """
    Reads the data and splits it into tokens and creates paddings masks
    
    parameters:
    --------------------------
    data: of the form [(model_answer, student_answer, score) ... ]
    add, mul: convert the score to class id by (score+add)*mul
    
    For example: if the score is 3.5 on a scale of 1 to 5, it is assigned the class id (3.5+(-1))*2 = 5
    """
    add = 0.0
    mul = 2.0
    xa1=[]
    xb1=[]
    cachedStopWords = stopwords.words('english')

    y2=[]
    y = []
    for i in range(0,len(data)):
        xa1.append(data[i][0].lower())
        xb1.append(data[i][1].lower())
        score = (float(data[i][2])+add)*mul
        y.append(float(data[i][2]))
        y2.append(int(score))

    lengths=[]
    xa12 = []
    xb12 = []
    for i in xa1:
        q = [x.lower() for x in i.split()]
        lengths.append(len(q))
        xa12.append(" ".join(q))
        
    for i in xb1:
        q = [x.lower() for x in i.split()]
        lengths.append(len(q))
        xb12.append(" ".join(q))
        
    maxlen = numpy.max(lengths)
    emb1,mas1=get_masks(xa12,maxlen)
    emb2,mas2=get_masks(xb12,maxlen)
    
    y2=np.array(y2,dtype=np.int32)
    y = np.array(y,dtype=np.float32)
    return emb1,mas1,emb2,mas2,y,y2

def get_masks(xa,maxlen):
    """
    Given a string and an integer maxlen, this function returns tokens and padding masks. 
    """
    n_samples = len(xa)
    ls=[]
    x_mask = numpy.zeros((maxlen, n_samples)).astype(np.float32)
    for i in range(0,len(xa)):
        q=xa[i].strip().split()
        if len(q) == 0:
            x_mask[0][i]=1
        for j in range(0,len(q)):
            x_mask[j][i]=1
        while(len(q)<maxlen):
            q.append(',')
        ls.append(q)
    xa=np.array(ls)
    return xa,x_mask

dtr=pickle.load(open("dwords.p",'rb'))
#dtr is dict of words mapped to a normalized form. This list is taken from "https://github.com/aditya1503/Siamese-LSTM"
#This is to take into account some word forms which do not appear in word2vec, for example "to" is mapped to "To"
#in dtr, since "to" doesn't exist word2vec
def embed(stmx):
    """
    Given a list of words, this function returns the word2vec embeddings of the list of words. 
    
    parameters:
    ----------------------------------
    stmx: list of words
    """
    #stmx=stmx.split()
    dmtr=numpy.zeros((stmx.shape[0],300),dtype=np.float32)
    count=0
    
    if stmx[0] == ',': #',' denotes padding
        dmtr[0] = numpy.full((300,),float(sys.maxint), dtype=np.float32)
        return dmtr
    
    while(count<len(stmx)):
        if stmx[count]==',':
            count+=1
            continue
        if stmx[count] in dtr: #if the word occurs in dtr
            if dtr[stmx[count]] in model: 
                dmtr[count]=model[dtr[stmx[count]]]
                count+=1
        	continue
        if stmx[count] in model: #if the word doesn't exist in dtr but exists in word2vec vocab
            dmtr[count]=model[stmx[count]]
        count+=1
    return dmtr


In [3]:
"""
Defining some utility functions
"""
def numpy_floatX(data):
    return np.asarray(data, dtype=config.floatX)

#unzip shared theano variables
def unzip(zipped):
    new_params = OrderedDict()
    for kk, vv in zipped.iteritems():
        new_params[kk] = vv.get_value()
    return new_params

#convert numpy arrays to theano shared variables
def init_tparams(params):
    tparams = OrderedDict()
    for kk, pp in params.iteritems():
        tparams[kk] = theano.shared(params[kk], name=kk)
    return tparams

dtr=pickle.load(open("dwords.p",'rb'))
prefix='lstm'
noise_std=0.
use_noise = theano.shared(numpy_floatX(0.))
flg=1
options=locals().copy()

In [4]:
#get model layer by name
def get_layer(name):
    fns = layers[name]
    return fns

#generate random normal matrix of size n1 x n2 with mean and standard deviation as given
def genm(mu,sigma,n1,n2):
    #return np.ones((n1,n2))
    return np.random.normal(mu,sigma,(n1,n2))


def getlayerx(d,pref,n,nin):
    """
    create shared variables (U, W, b) for the lstm as given in https://en.wikipedia.org/wiki/Long_short-term_memory#Traditional_LSTM
    
    parameters:
    --------------------------------
    d: OrderedDict containing information about all the layers in the network
    pref: name of the LSTM (we create 4 LSTMs, one fw and bw for each Siamese biLSTM)
    nin: Input dimension
    n: hidden dimension
    """
    mu=0.0
    sigma=0.2
    U = np.concatenate([genm(mu,sigma,n,n),genm(mu,sigma,n,n),genm(mu,sigma,n,n),genm(mu,sigma,n,n)])/np.sqrt(n)
    U=np.array(U,dtype=np.float32)
    W =np.concatenate([genm(mu,sigma,n,nin),genm(mu,sigma,n,nin),genm(mu,sigma,n,nin),genm(mu,sigma,n,nin)])/np.sqrt(np.sqrt(n*nin))
    W=np.array(W,dtype=np.float32)

    # if pref = "1lstm1', three shared variables by the name of 1lstm1_U, 1lstm1_W, and 1lstm1_b will be created
    d[_p(pref, 'U')] = U
    b = np.random.uniform(-0.5,0.5,size=(4*n,))
    b[n:n*2]=1.5
    d[_p(pref, 'W')] = W
    d[_p(pref, 'b')] = b.astype(config.floatX)
    return d

def creatrnnx():
    """
    Creates all the shared variables for 4 lstms as required. Input dimension is 300 and embedding dimension is 50
    """
    newp=OrderedDict()
    newp=getlayerx(newp,'1lstm1',50,300)
    newp=getlayerx(newp,'1lstm2',50,300)
    newp=getlayerx(newp,'2lstm1',50,300)
    newp=getlayerx(newp,'2lstm2',50,300)
    return newp


def dropout_layer(state_before, use_noise, rrng,rate):
    proj = tensor.switch(use_noise,
                         (state_before *rrng),
                         state_before * (1-rate))
    return proj

def getpl2(prevlayer,pre,mymask,used,rrng,size,tnewp):
    proj = lstm_layer2(tnewp, prevlayer, options,
                                        prefix=pre,
                                        mask=mymask,nhd=size)
    if used:
        print "Added dropout"
        proj = dropout_layer(proj, use_noise, rrng,0.5)

    return proj

def lstm_layer2(tparams, state_below, options, prefix='lstm', mask=None,nhd=None):
    """
    Create LSTM layer
    
    parameters:
    --------------------------------
    tparams: shared variables defined using creaternnx()

    """
    nsteps = state_below.shape[0]
    if state_below.ndim == 3:
        n_samples = state_below.shape[1]
    else:
        n_samples = 1
    assert mask is not None

    def _slice(_x, n, dim):
            if _x.ndim == 3:
                return _x[:, :, n * dim:(n + 1) * dim]
            return _x[:, n * dim:(n + 1) * dim]

    def _step(m_, x_, h_, c_):
            preact = tensor.dot(h_, tparams[_p(prefix, 'U')].T)
            preact += x_
            preact += tparams[_p(prefix, 'b')]
            i = tensor.nnet.sigmoid(_slice(preact, 0, nhd))
            f = tensor.nnet.sigmoid(_slice(preact, 1, nhd))
            o = tensor.nnet.sigmoid(_slice(preact, 2, nhd))
            c = tensor.tanh(_slice(preact, 3, nhd))
            c = f * c_ + i * c
            c = m_[:, None] * c + (1. - m_)[:, None] * c_
            h = o * tensor.tanh(c)
            h = m_[:, None] * h + (1. - m_)[:, None] * h_
            return [h, c]

    state_below = (tensor.dot(state_below, tparams[_p(prefix, 'W')].T) + tparams[_p(prefix, 'b')].T)
    #print "hvals"
    dim_proj = nhd
    [hvals,yvals], updates = theano.scan(_step,
                                sequences=[mask, state_below],
                                outputs_info=[tensor.alloc(numpy_floatX(0.),
                                                           n_samples,
                                                           dim_proj),
                                              tensor.alloc(numpy_floatX(0.),
                                                           n_samples,
                                                           dim_proj)],
                                name=_p(prefix, '_layers'),
                                n_steps=nsteps)
    return hvals

In [5]:
class lstm():    
    """
    This clss creates the proposed model and defines the train and eval methods
    """
    def save_model(self, name):
        """
        Save the model parameters as a pickle dump
        """
        with open(name,'wb') as f:
           pickle.dump(unzip(self.tnewp), f)
        
    def __reverse(self, emb, mask):
        """
        Reverse the input tensor `emb` for creating BiLSTM
        Parameters:
        emb: The input tensor
        mask: the mask vector of the corresponding tensor (used for padding)
        """
        source = emb.dimshuffle(1, 0, 2)
        sz = mask.sum(axis=0).astype('int64')
        
        def _step(psource, psz):
            psz = tensor.switch(psz>0, psz, 1)
            return tensor.concatenate([psource[psz-1::-1, :], tensor.zeros((mask.shape[0]-psz, 300))] )
        
        outputs, updates = theano.scan(
            fn=_step,
            sequences=[source, sz]
            )
        
        return tensor.cast(outputs.dimshuffle(1, 0, 2), theano.config.floatX)
    
    def create_emd(self, idx, emb11, mask11, rev_emb11, emb21, mask21, rev_emb21):
        """
        Given the input embeddings for both model and student answer, create the distance matrix as described
        paper
        
        parameters:
        ---------------------------------------
        idx:
        emb11: embeddings of words of model answer
        mask11: mask for model answer:
        rev_emb11: reverse of emb11
        emb21: embeddings of words of student answer
        mask21: mask for student answer:
        rev_emb21: reverse of emb21
        """
        trng = RandomStreams(1234)
        use_noise = theano.shared(numpy_floatX(0.))

        rate=0.5
        rrng=trng.binomial(emb11.shape,p=1-rate, n=1,dtype=emb11.dtype)

        #create LSTMs
        proj11_all=getpl2(emb11,str(idx)+'_1lstm1',mask11,False,rrng,50,self.tnewp)
        proj12_all=getpl2(rev_emb11,str(idx)+'_1lstm2',mask11,False,rrng,50,self.tnewp)
        proj21_all=getpl2(emb21,str(idx)+'_2lstm1',mask21,False,rrng,50,self.tnewp)
        proj22_all=getpl2(rev_emb21,str(idx)+'_2lstm2',mask21,False,rrng,50,self.tnewp)
        
        #concatenate the fw and bw hidden states
        proj1_all = T.concatenate([proj11_all, proj12_all], axis=2)
        proj2_all = T.concatenate([proj21_all, proj22_all], axis=2)
        
        #distance matrix calculation based on L1 distance
        distance_matrix = ( T.sum( T.abs_( proj1_all.dimshuffle(0,'x',1,2)-proj2_all.dimshuffle('x',0,1,2)), axis=3) )
        return distance_matrix
    
    def __init__(
                    self, 
                    nam, 
                    load=False,
                    load_emd=False, 
                    training=False, 
                    num_classes=11,
                    num_emds=1
                ):
        """
        Parameters:
        nam: name of the model file
        load: load the model if True
        load_emd: if True, load all the parameters except the SVOR layer. if `load` is False, this is not executed
        training: True, if want to train the model else gradients are not computed
        num_classes: number of classes in the ordinal layer, default is 11 for Mohler
        num_emds: number of emds to calculate. We experimented with having multiple different kinds of distance matrices, 
                  but settled on using only one in our final experiments
        """
        self.newp = OrderedDict()
        for idx in range(num_emds):
            newp_=creatrnnx()
            for i in newp_.keys():
                if i[0]=='1':
	                newp_['2'+i[1:]]=newp_[i]

	        for  i in newp_.keys():
	        	self.newp[str(idx)+"_"+i] = newp_[i]
        #We assign the same value to the shared variables of both the BiLSTMs as the weights are tied in Siamese Networks
                
        self.num_emds = num_emds
        self.tnewp = OrderedDict()
        y = tensor.ivector('y')
        
        #tensors and masks for model and student answers
        mask11 = tensor.matrix('mask11', dtype=config.floatX)
        mask21 = tensor.matrix('mask21', dtype=config.floatX)
        emb11=theano.tensor.ftensor3('emb11')
        emb21=theano.tensor.ftensor3('emb21')
        
        #reverse tensors
        rev_emb11 = self.__reverse(emb11, mask11)
        rev_emb21 = self.__reverse(emb21, mask21)

        #parameters for SVOR
        self.newp['FU'] = np.random.normal(-1.0,0.5,(num_emds, 1)).astype(config.floatX)
        self.newp['Fb'] = np.zeros((num_classes-1,)).astype(config.floatX)

       	if load:
           	newp = pickle.load(open(nam, 'rb'))
        	self.newp=init_tparams(newp)

        self.tnewp = init_tparams(self.newp)

        #flows matrix for EMD
        flows_list = []
        emds = []
        self.f_dm = []
        for i in range(self.num_emds):
        	flow = theano.tensor.ftensor3('flow_'+str(i))
        	flows_list.append(flow)

        	distance_matrix = self.create_emd(i, emb11, mask11, rev_emb11, emb21, mask21, rev_emb21)
        	emd = T.sum(flow*distance_matrix, axis=[0,1])
        	emd = emd.reshape((-1,1))
        	emds.append(emd)

        	self.f_dm.append(theano.function([emb11,mask11,emb21,mask21], distance_matrix, allow_input_downcast=True))

        features = T.concatenate(emds, axis=1)
        flows = T.stack(flows_list, axis=0)
        
        #define the prediction function and losses
        pred = T.sum( T.switch(T.dot(features, self.tnewp['FU']).flatten().dimshuffle(0,'x')-self.tnewp['Fb'].dimshuffle('x',0) >= 0, 1, 0) , axis=1)/5.0
        pred_dist = T.dot(features, self.tnewp['FU']).flatten().dimshuffle(0,'x')-self.tnewp['Fb'].dimshuffle('x',0)
        
        cost = sum([T.sum(T.switch( y <= j , T.maximum(0, 1 + (pred_dist.T)[j]), T.maximum(0, 1 - (pred_dist.T)[j]))) for j in range(num_classes-1)])
        cost += 0.5*T.sum(T.dot(self.tnewp['FU'], self.tnewp['FU'].T)) #regularization
        
        lr = tensor.scalar(name='lr')
        
        #L-x regularization, didn't help in final results
        l2_weight = 0.000
        l1_weight = 0.000
        l1_penalty = sum([T.sum(abs(temp)) for temp in self.tnewp.values()])
        l2_penalty = sum([T.sum(temp**2) for temp in self.tnewp.values()])
        cost += l2_weight*l2_penalty + l1_weight*l1_penalty
 
        # defining theano functions for useful tensors
        self.f_emds = theano.function([emb11,mask11,emb21,mask21]+flows_list, features, allow_input_downcast=True)    
        self.f_pred_dist = theano.function([emb11,mask11,emb21,mask21]+flows_list,pred_dist,allow_input_downcast=True)
        self.f_pred = theano.function([emb11, mask11,emb21,mask21]+flows_list,pred,allow_input_downcast=True)
        self.f_cost = theano.function([emb11,mask11,emb21,mask21,y]+flows_list,cost,allow_input_downcast=True)

        if training==True:
            #compute gradients and assign the average of the parameter values to both LSTMs
            gradi = tensor.grad(cost, wrt=self.tnewp.values())#/bts
            grads=[]
            l=len(gradi)-2
            for i in range(0,l/2):
                gravg=(gradi[i]+gradi[i+l/2])/(2.0)
                grads.append(gravg)
            for i in range(0,len(self.tnewp.keys())/2-1):
                    grads.append(grads[i])
            grads.append(gradi[l])
            grads.append(gradi[l+1])

            self.f_grad_shared, self.f_update = adadelta(lr, self.tnewp, grads,emb11,mask11,emb21,mask21,y,cost,flows_list)

    def solve_emd(self, dm, b, d):
        """
        slow EMD calculation using scipy's linprog. Not used in final experiments
        Parameters:
        ------------------
        dm: distance matrix
        b: model mask
        d: answer mask
        """
        dm = dm.swapaxes(1,2)
        dm = dm.swapaxes(0,1)
        l = dm.shape[1]
        flows = []
        
        for ii in range(dm.shape[0]):
            bs = b.sum(axis=0).astype(int)
            ds = d.sum(axis=0).astype(int)
            c = dm[ii][:bs[ii],:ds[ii]].flatten()
            A_eq = []
            
            for j in range(ds[ii]):
                row = [0 for k in range(bs[ii]*ds[ii])]
                for k in range(j*bs[ii], (j+1)*bs[ii]):
                    row[k] = 1
                A_eq.append(row)
                
            for j in range(bs[ii]):
                row = [0 for k in range(bs[ii]*ds[ii])]
                for k in range(0, bs[ii]*ds[ii], ds[ii]):
                    row[k] = 1
                A_eq.append(row)
            b_eq = [1.0/bs[ii] for j in range(bs[ii])]+[1.0/ds[ii] for j in range(ds[ii])]
            A_eq = np.array(A_eq)
            b_eq = np.array(b_eq)
            r = linprog(c, A_eq = A_eq, b_eq=b_eq, options=dict(maxiter=10000))
            df = r.x.reshape((bs[ii],ds[ii]))
            f = np.zeros((l,l))
            f[:bs[ii], :ds[ii]] = df
            flows.append(f)
        flows = np.array(flows, dtype=np.float32)
        flows = flows.swapaxes(0,1)
        flows = flows.swapaxes(1,2)
        return flows

    def solve_emd_fast(self, dm, b, d, lambd=0.1):
        """
        fast and approximate EMD calculation using Sinkhorn, the lambda used here is the reciprocal of lambda described in the sinkhorn paper
        """
        import ot
        dm = dm.swapaxes(1,2)
        dm = dm.swapaxes(0,1)
        bs = b.sum(axis=0).astype(int)
        ds = d.sum(axis=0).astype(int)
        l = dm.shape[1]
        #print l
        flows = []
        for ii in range(dm.shape[0]):
            #print bs[ii], ds[ii]
            if ds[ii] == 0:
                ds[ii] = 1
            c = dm[ii][:bs[ii], :ds[ii]]

            s1 = np.array([1.0/bs[ii] for kk in range(bs[ii])])
            s2 = np.array([1.0/ds[ii] for kk in range(ds[ii])])
            df = ot.emd(s1, s2, c)
            f = np.zeros((l, l))
            f[:bs[ii], :ds[ii]] = df
            flows.append(f)
        flows = np.array(flows, dtype=np.float32)
        flows = flows.swapaxes(0,1)
        flows = flows.swapaxes(1,2)
        return flows
    
    #training the LSTM
    def train_lstm(
                    self,
                    train,
                    test,
                    max_epochs=10,
                    showtest=True,
                    batchsize=32
                ):
        """
        Parameters
        -----------------------
        train: training dataset, list of (model, student, score), score lie in [0,5]
        max_epochs: maximum number of epochs
        test: test dataset, same format as training
        max_epochs: maximum number of epochs to train
        showtest: if True, show test performance after every epoch
        batchsize: size of each batch to process at once.
        """
        print "Training"
        crer=[]
        cr=1.6
        freq=0
        dfreq=40#display frequency
        valfreq=800# Validation frequency
        lrate=0.0001
        precision=2
        
        for eidx in xrange(0,max_epochs):
            sta=time.time()
            num=len(train)
            nd=eidx
            sta=time.time()
            
            print 'Epoch',eidx
            rnd=sample(xrange(len(train)),len(train))
            
            for i in range(0,num,batchsize):
                ste = time.time()
                q=[]
                x=i+batchsize
                if x>num:
                    x=num
                for z in range(i,x):
                    q.append(train[rnd[z]])
                    
                #prepare_data reads the inputs of the form [(model, student, score) ... ] and returns 
                #x1, x2: tokenized answers. mas1, mas2: masks of both answers. y2: class names, y: score values
                x1, mas1, x2, mas2, y, y2 = prepare_data(q)
                ls=[]
                ls2=[]
                freq+=1
                use_noise.set_value(1.)
                
                #convert words to word2vec embeddings.
                for j in range(0,len(x1)):
                    ls.append(embed(x1[j]))
                    ls2.append(embed(x2[j]))
                trconv=np.dstack(ls)
                trconv2=np.dstack(ls2)
                emb2=np.swapaxes(trconv2,1,2)
                emb1=np.swapaxes(trconv,1,2)

                #alternating optimization
                #first calculate flow values, then backprop to update parameters
                flows = []
                for ei in range(self.num_emds):
                	dm = self.f_dm[ei](emb1, mas1, emb2, mas2)
                	flows.append(self.solve_emd_fast(dm, mas1, mas2))

                cst=self.f_grad_shared(*([emb2, mas2, emb1, mas1, y2] + flows))
                s=self.f_update(lrate)
                
                
                if np.mod(freq,dfreq)==0:
                    print 'Epoch ', eidx, 'Update ', freq, 'Cost ', cst
                ete = time.time()
                
            sto=time.time()
            print "epoch took:",sto-sta
            
        if showtest:
            print "Test error -",self.get_error(test)
    def get_error(self, data):
        """
        Returns MSE, MAE, Pearson and Spearman scores
        Parameters:
        ---------------------
        data: list of (model, student, score)
        """
        count=[]
        num=len(data)
        px=[]
        yx=[]
        use_noise.set_value(0.)
        for i in range(0,num,256):
            q=[]
            x=i+256
            if x>num:
                x=num
            for j in range(i,x):
                q.append(data[j])
            x1, mas1, x2, mas2, y, y2 = prepare_data(q)#y is actual score, y2 is class number
            
            ls=[]
            ls2=[]
            for j in range(0,len(q)):
                ls.append(embed(x1[j]))
                ls2.append(embed(x2[j]))
            trconv=np.dstack(ls)
            trconv2=np.dstack(ls2)
            emb2=np.swapaxes(trconv2,1,2)
            emb1=np.swapaxes(trconv,1,2)

            flows = []
            for ei in range(self.num_emds):
            	dm = self.f_dm[ei](emb1, mas1, emb2, mas2)
            	flows.append(self.solve_emd_fast(dm, mas1, mas2))

            pred=self.f_pred(*([emb1, mas1, emb2, mas2] + flows))
            
            for z in range(0,len(q)):
                yx.append(y[z])
                px.append(pred[z])
        px=np.array(px)
        yx=np.array(yx)
        return np.mean((px-yx)**2), np.mean(abs(px-yx)),meas.pearsonr(px,yx)[0],meas.spearmanr(yx,px)[0]


In [6]:
def adadelta(lr, tparams, grads, emb11,mask11,emb21, mask21, y, cost, flows):
    """
    We use adadelta optimizer during training
    """
    zipped_grads = [theano.shared(p.get_value() * numpy_floatX(0.),
                                  name='%s_grad' % k)
                    for k, p in tparams.iteritems()]
    running_up2 = [theano.shared(p.get_value() * numpy_floatX(0.),
                                 name='%s_rup2' % k)
                   for k, p in tparams.iteritems()]
    running_grads2 = [theano.shared(p.get_value() * numpy_floatX(0.),
                                    name='%s_rgrad2' % k)
                      for k, p in tparams.iteritems()]
    zgup = [(zg, g) for zg, g in zip(zipped_grads, grads)]
    rg2up = [(rg2, (0.95 * rg2 + 0.05* (g ** 2)))
             for rg2, g in zip(running_grads2, grads)]
    f_grad_shared = theano.function([emb11,mask11,emb21,mask21,y]+flows, cost, updates=zgup + rg2up,
                                    name='adadelta_f_grad_shared')
    updir = [-tensor.sqrt(ru2 + 1e-6) / tensor.sqrt(rg2 + 1e-6) * zg
             for zg, ru2, rg2 in zip(zipped_grads,
                                     running_up2,
                                     running_grads2)]
    ru2up = [(ru2, (0.95 * ru2 + 0.05 * (ud ** 2)))
             for ru2, ud in zip(running_up2,updir)]
    param_up = [(p, p + ud) for p, ud in zip(tparams.values(), updir)]
    f_update = theano.function([lr], [], updates=ru2up + param_up,
                               on_unused_input='ignore',
                               name='adadelta_f_update')
    return f_grad_shared, f_update

In [7]:
#create folders to save trained models
import os
all_results = []
print os.getcwd()

results_file = open('results.txt','w')

if not os.path.exists("./trained_models"):
	os.mkdir("./trained_models")
	
if not os.path.exists("./trained_models/mohler"):
	os.mkdir("./trained_models/mohler")

/home/sachin/Experiments/codebase/neural-asag/code


In [ ]:
#run 12 fold training
for i in range(12):
    #create an instance of lstm class
	sls=lstm("",load=False,training=True)
    #load training and test data
	train, test = pickle.load(open('data/mohler/'+str(i)+".fold", 'rb'))

    #start training
	sls.train_lstm(train, test)
	results = sls.get_error(test)
	print results

	all_results.append(results)
	results_file.write(str(results)+"\n")
	sls.save_model('./trained_models/mohler/'+str(i)+'.p')

In [ ]:
#print mean of all results
print numpy.mean(all_results, axis=0)
results_file.close()